In [1]:
import numpy as np
from PIL import Image
import pandas as pd
import shutil
import time
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import cv2
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Now Take All the ata into a list
data_dir='C:\\Users\\MVI_lab_2\\Desktop\\piline\\dataset\\brain\\'
image_dir='C:\\Users\\MVI_lab_2\\Desktop\\piline\\dataset\\ndbt\\'
total_image=3064
trainindata=[]
for i in range(1,total_image+1):
  filename=str(i)+".mat"
  data=h5py.File(os.path.join(data_dir,filename),"r")
  trainindata.append(data)

In [3]:
import numpy as np
 # Now take all the image as train and test
trainx=[]
trainy=[]
testx=[]
testy=[]
valx=[]
valy=[]
imx = []
imy =[]
imz =[]

# For trainx and trainy
for i in range(2451):
  image=trainindata[i]["cjdata"]["image"][()]
  if image.shape==(512,512):
    imx.append(str(i+1))
    label=int(trainindata[i]["cjdata"]["label"][()])-1
    trainy.append(str(label))
    
# For trainx and trainy
for i in range(2451,2757):
  image=trainindata[i]["cjdata"]["image"][()]
  if image.shape==(512,512):
    imy.append(str(i+1))
    label=int(trainindata[i]["cjdata"]["label"][()])-1
    testy.append(str(label))

for i in range(2757,total_image):
  image=trainindata[i]["cjdata"]["image"][()]
  if image.shape==(512,512):
    imz.append(str(i+1))
    label=int(trainindata[i]["cjdata"]["label"][()])-1
    valy.append(str(label))

In [4]:
for i in imx:
    im_name = i + ".jpg"
    im_path = os.path.join(image_dir,im_name)
    trainx.append(im_path)

# For trainx and trainy
for i in imy:
    im_name = i + ".jpg"
    im_path = os.path.join(image_dir,im_name)
    testx.append(im_path)

for i in imz:
    im_name = i + ".jpg"
    im_path = os.path.join(image_dir,im_name)
    valx.append(im_path)

In [5]:
print(len(trainx),len(trainy))

2436 2436


In [6]:
train_dict = {'image': trainx, 'label': trainy}
train_df = pd.DataFrame(train_dict)
test_dict = {'image': testx, 'label': testy}
test_df = pd.DataFrame(test_dict)
val_dict = {'image': valx, 'label': valy}
val_df = pd.DataFrame(val_dict)

In [7]:
t_df = pd.concat([train_df, test_df, val_df], axis=0)

In [9]:
from sklearn.model_selection import train_test_split
train_split=.8
test_split=.1
dummy_split=test_split/(1-train_split)
train_df, dummy_df=train_test_split(t_df, train_size=train_split, shuffle=True, random_state=123)
test_df, val_df=train_test_split(dummy_df, train_size=dummy_split, shuffle=True, random_state=123)
print ('train_df length: ', len(train_df), ' test_df length: ', len(test_df), '  valid_df length: ', len(val_df))

train_df length:  2439  test_df length:  305   valid_df length:  305


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
height=512
width=512
channels=3
batch_size=32
img_shape=(height, width, channels)
img_size=(height, width)
def scalar(img):
    return img/127.5-1  # scale pixel between -1 and +1
gen=ImageDataGenerator(preprocessing_function=scalar)
train_gen = gen.flow_from_dataframe(train_df, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                    color_mode='grayscale', shuffle=True, batch_size=batch_size)
test_gen = gen.flow_from_dataframe(test_df, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                    color_mode='grayscale', shuffle=True, batch_size=batch_size)
val_gen = gen.flow_from_dataframe(val_df, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                    color_mode='grayscale', shuffle=True, batch_size=batch_size)
classes=list(train_gen.class_indices.keys())
class_count=len(classes)

Found 2439 validated image filenames belonging to 3 classes.
Found 305 validated image filenames belonging to 3 classes.
Found 305 validated image filenames belonging to 3 classes.


In [11]:
from collections import Counter
counter = Counter(train_gen.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
class_weights

{2: 1.6162464985994398, 1: 1.0, 0: 2.021015761821366}

In [13]:
#Model building starts
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
import numpy as np
tf.keras.backend.clear_session()

In [14]:
##Initial  BLock of the model
ini_input=keras.Input(shape=(512,512,1),name="image")

x1=layers.Conv2D(64,(22,22),strides=2)(ini_input)
x1=layers.MaxPooling2D((4,4))(x1)
x1=layers.BatchNormalization()(x1)

x2=layers.Conv2D(128,(11,11),strides=2,padding="same")(x1)
x2=layers.MaxPooling2D((2,2))(x2)
x2=layers.BatchNormalization()(x2)

x3=layers.GlobalAveragePooling2D()(x2)
x3=layers.Activation("relu")(x3)

x4=layers.Dense(512,"relu")(x3)
x4=layers.BatchNormalization()(x4)
x5=layers.Dense(256,"relu")(x4)
x5=layers.BatchNormalization()(x5)
x5=layers.Dropout(.2)(x5)
x6=layers.Dense(3)(x5)
pred=layers.Activation("softmax")(x6)

model=keras.Model(inputs=ini_input,outputs=pred)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 512, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 246, 246, 64)      31040     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 61, 61, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 128)       991360    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 128)      0     

In [16]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
x_train = []
y_train = []
for x, y in train_gen:
    x_train.append(x)
    y_train.append(y)

x_test = []
y_test = []
for x, y in train_gen:
    x_test.append(x)
    y_test.append(y)

In [ ]:
r=model.fit(x_train,
            y_train,
          epochs=100,
          batch_size=32,
          verbose=1,
          validation_data=(x_test, y_test),
          shuffle=False
          )